In [1]:
import pandas as pd
from wmfdata import hive, charting
from wmfdata.charting import comma_fmt, pct_fmt
from wmfdata.utils import pct_str

## Read and clean data

In [2]:
#final interim frame (with new and stub articles)
interim_frame_updated = pd.read_csv("../../data/processed/query_results/content_quality/interim_frame_updated.csv", sep=',', encoding = 'utf-8', parse_dates=True) 

#rename column
interim_frame_updated.rename(columns={'wikicode':'database_code'}, inplace = True)

In [3]:
interim_frame_updated_to_hive = interim_frame_updated[['database_code','article_type','page_id', 'page_title']]

In [4]:
#count num of non zero values
interim_frame_updated_to_hive.astype(bool).sum(axis=0)

database_code    8863
article_type     8863
page_id          8863
page_title       8863
dtype: int64

In [5]:
#id zeros
interim_frame_updated_to_hive.loc[interim_frame_updated_to_hive['page_id']==0]

,database_code,article_type,page_id,page_title


In [6]:
#count nans
interim_frame_updated_to_hive.isna().sum()

database_code    0
article_type     0
page_id          0
page_title       0
dtype: int64

In [7]:
#remove null page_id rows
interim_frame_updated_to_hive = interim_frame_updated_to_hive.dropna(axis=0, subset=['page_id'])

In [8]:
interim_frame_updated_to_hive.isnull().sum().sum()

0

## Preparing to create table and load into hive

In [9]:
interim_frame_updated_to_hive.to_csv("../../data/processed/query_results/content_quality/interim_frame_updated_to_hive.csv", sep=',', encoding = 'utf-8', index=False)

In [10]:
articles = "../../data/processed/query_results/content_quality/interim_frame_updated_to_hive.csv"
hive.load_csv(
    articles,
    field_spec="database_code string, article_type string, page_id int, page_title string",
    db_name="florez",
    table_name="glow_tiger_articles",
    sep=','
)